# URBANopt to Alfalfa workflow 

In [59]:
import os
import shutil
import json
import pandas
import datetime
import time
from alfalfa_client.alfalfa_client import AlfalfaClient
from pprint import pprint
from pathlib import Path


## Define alfalfa client object

In [72]:
ac = AlfalfaClient(url='https://cctwin.nrel.gov')

## Define Alfalfa simulation parameters

In [61]:
# If you are using historian, you will need to search for this time period in Grafana dashboard to view results.
start_dt = datetime.datetime(2021, 7, 1, 12, 2, 0)
end_dt = datetime.datetime(2021, 7, 3, 0, 0, 0)

# For external_clock == true, API calls are used to advance the model.  
# If external_clock == false, Alfalfa will handle advancing the model according to a specified timescale (timescale 1 => realtime)
params = {
    "external_clock": True,
    "start_datetime": start_dt,
    "end_datetime": end_dt
}

## Create zipped folders to upload to Alfalfa from the URBANopt Project

The code below will create zipped folders for each URBANopt building model that can be uploaded to Alfalfa. The Zipped folder contains: 

    - Model Folder: Contains OpenStudio model (.osm file) for each building created using the URBANopt simulation.
    - Measures Folder: Measures to be added while running Alfalfa in the OpenStudio Workflow
    - Weather Folder: Contains EPW weather file 
    - workflow.osw file: OpenStudio workflow file

**Define the following variables before running the code**:

- `uo_folder` : URBANopt Scenario Directory 
- `weather`: Define weather file used in URBANopt project
- `workflow`: OpenStudio workflow file name

In [86]:
# Set URBANopt scenario directory
uo_folder = 'C:/a/baseline_stochastic/run/baseline_scenario_stochastic'

uo_scenario_folder = uo_folder.split("/")[-1]
uo_scenario_path = os.path.join('./{}'.format(uo_scenario_folder))
Path(uo_scenario_path).mkdir(parents=True, exist_ok=True)

print("Zipped folders saved at {}".format(uo_scenario_path))

# Add .epw filename
weather = "USA_CO_Denver.Intl.AP.725650_TMY3.epw"

# Add .osw filename
workflow = "workflow.osw"

for filename in os.listdir(uo_folder):
        
    if os.path.isfile(os.path.join(uo_folder, filename, 'in.osm').replace("\\","/")):
        uo_model_filepath = os.path.join(uo_scenario_path, filename)
        Path(uo_model_filepath).mkdir(parents=True, exist_ok=True)
        Path(os.path.join(uo_model_filepath, 'models')).mkdir(parents=True, exist_ok=True)
        Path(os.path.join(uo_model_filepath, 'measures')).mkdir(parents=True, exist_ok=True)
        Path(os.path.join(uo_model_filepath, 'weather')).mkdir(parents=True, exist_ok=True)

        shutil.copy((os.path.join(uo_folder,filename, 'in.osm')), (os.path.join(uo_model_filepath, 'models', '{}.osm'.format(filename))))
        shutil.copy((os.path.join(uo_folder, '../../weather', weather)), (os.path.join(uo_model_filepath, 'weather')))
        
        osw = {"seed_file": "{}.osm".format(filename),
           "weather_file": "{}".format(weather),
           "run_directory": "./run/",
           "file_paths": [
               "./weather/",
               "./models/"
           ]
          }
        
        f = open(os.path.join(uo_model_filepath, workflow), "w+")
        f.write(json.dumps(osw, indent=4))
        f.close()
        
        shutil.make_archive((uo_model_filepath), 'zip', (uo_model_filepath))
        shutil.rmtree(uo_model_filepath)
        
        model = os.path.join('./{}/{}.zip'.format(uo_scenario_folder, filename)).replace("\\","/")
        print(model)

print('done')       

Zipped folders saved at ./baseline_scenario_stochastic
./baseline_scenario_stochastic/1.zip
./baseline_scenario_stochastic/10.zip
./baseline_scenario_stochastic/100.zip
./baseline_scenario_stochastic/101.zip
./baseline_scenario_stochastic/102.zip
./baseline_scenario_stochastic/103.zip
./baseline_scenario_stochastic/104.zip
./baseline_scenario_stochastic/105.zip
./baseline_scenario_stochastic/106.zip
./baseline_scenario_stochastic/107.zip
./baseline_scenario_stochastic/108.zip
./baseline_scenario_stochastic/109.zip
./baseline_scenario_stochastic/11.zip
./baseline_scenario_stochastic/110.zip
./baseline_scenario_stochastic/111.zip
./baseline_scenario_stochastic/112.zip
./baseline_scenario_stochastic/113.zip
./baseline_scenario_stochastic/114.zip
./baseline_scenario_stochastic/115.zip
./baseline_scenario_stochastic/116.zip
./baseline_scenario_stochastic/117.zip
./baseline_scenario_stochastic/118.zip
./baseline_scenario_stochastic/119.zip
./baseline_scenario_stochastic/12.zip
./baseline_sce

## Upload models to Alfalfa

- Submit models to Alfalfa server
- Print model inputs (EP Actuators)
- Print model outputs (EP Sensors)

In [78]:
 for filename in os.listdir(uo_scenario_path):
        model = os.path.join(uo_scenario_path, filename)
        print(model)
        
        model_id = ac.submit(model)
        print(model_id)
        
        # Start Simulations
        print(f"Starting site: {model_id}")
        ac.start(model_id, **params)
        
        # Get model input points
        print(f"{model_id} inputs:")
        pprint(ac.inputs(m1_id))
        

./models_urbanopt\1.zip
Desired status: READY		Current status: PREPROCESSING
Desired status: READY		Current status: PREPROCESSING
c2ddb440-cf56-11ed-9bda-7dbdbc75d29b
Starting site: c2ddb440-cf56-11ed-9bda-7dbdbc75d29b
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED


AlfalfaException: Traceback (most recent call last):
  File "/alfalfa/alfalfa_worker/jobs/openstudio/step_run.py", line 86, in init_sim
    [self.ep.status, self.ep.msg] = self.ep.accept_socket()
  File "/usr/local/lib/python3.8/dist-packages/mlep/mlep_process.py", line 162, in accept_socket
    (self.comm_socket, self.client_address) = self.server_socket.accept()
  File "/usr/lib/python3.8/socket.py", line 292, in accept
    fd, addr = self._accept()
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/alfalfa/alfalfa_worker/lib/job.py", line 104, in start
    self.exec()
  File "/alfalfa/alfalfa_worker/jobs/step_run_base.py", line 106, in exec
    self.init_sim()
  File "/alfalfa/alfalfa_worker/jobs/openstudio/step_run.py", line 88, in init_sim
    self.check_error_log()
  File "/alfalfa/alfalfa_worker/jobs/openstudio/step_run.py", line 398, in check_error_log
    raise JobExceptionExternalProcess(